# 將自己的 google drive 空間 mount 到 colab 環境

- 若在 colab 環境，請執行下面程式，點開連結及選取自己的 gmail，將許可碼拷貝貼在格子裡

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# 安裝相關套件

In [1]:
!pip install contextlib2
!pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
!pip install toolz --upgrade

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-install-3mdvdV/pycocotools
  Created wheel for pycocotools: filename=pycocotools-2.0-cp27-cp27mu-linux_x86_64.whl size=139135 sha256=2b4ffcf50476fb67c815830675290106782519348eeeb3b177869c9c19ef391e
  Stored in directory: /tmp/pip-ephem-wheel-cache-WZJ6UH/wheels/72/d6/19/422d4897cdeba15cf0f491a9353b556651a65da43f6510f84d
Successfully built pycocotools
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 49 kB 808 kB/s 
  Created wheel for toolz: filename=toolz-0.10.0-py2-none-any.whl size=55573 sha256=be23c508ed457940202fd736a56236e5bef33a140c67ac44c5a1cdbb02bd7187
  Stored in directory: /home/rico-li/.cache/pip/wheels/be/61/47/7646faed0f0cb2f31a08e02c394ba9d6c0b15dcd0d63db2686
Successfully built toolz


# 設定環境

- 由於 colab 每個 notebook 環境皆獨立，必須重新安裝套件

In [ ]:
%cd /content/gdrive/My\ Drive/models/research
!mkdir train eval
#%set_env PYTHONPATH=`pwd`:`pwd`/slim

In [ ]:
!python setup.py install

In [ ]:
%cd slim
!python setup.py install
%cd ..

# 下載資料 (pets) ; 將資料轉換成 tfrecord

- 除了 pets, 也可以下載 coco / open images / pascal VOC 資料集並用已經提供的 tfrecord 轉換工具作轉換
    - 轉換程式在 `object_detection/dataset_tools/` 裡
- 也可以用定義自己的資料集及資料格式，再客製轉換程式將資料轉換成 tfrecord

In [2]:
# 下載資料
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
# 解壓縮
!tar -xvf images.tar.gz
!tar -xvf annotations.tar.gz

er_84.png
annotations/trimaps/._wheaten_terrier_85.png
annotations/trimaps/wheaten_terrier_85.png
annotations/trimaps/._wheaten_terrier_86.png
annotations/trimaps/wheaten_terrier_86.png
annotations/trimaps/._wheaten_terrier_87.png
annotations/trimaps/wheaten_terrier_87.png
annotations/trimaps/._wheaten_terrier_88.png
annotations/trimaps/wheaten_terrier_88.png
annotations/trimaps/._wheaten_terrier_89.png
annotations/trimaps/wheaten_terrier_89.png
annotations/trimaps/._wheaten_terrier_9.png
annotations/trimaps/wheaten_terrier_9.png
annotations/trimaps/._wheaten_terrier_90.png
annotations/trimaps/wheaten_terrier_90.png
annotations/trimaps/._wheaten_terrier_91.png
annotations/trimaps/wheaten_terrier_91.png
annotations/trimaps/._wheaten_terrier_92.png
annotations/trimaps/wheaten_terrier_92.png
annotations/trimaps/._wheaten_terrier_93.png
annotations/trimaps/wheaten_terrier_93.png
annotations/trimaps/._wheaten_terrier_94.png
annotations/trimaps/wheaten_terrier_94.png
annotations/trimaps/._wh

In [11]:
import os 
!python /home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research/object_detection/dataset_tools/create_pet_tf_record.py \
 --label_map_path=/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research/object_detection/data/pet_label_map.pbtxt \
 --data_dir= /home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research \
 --output_dir= /home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research
!ls *.record*

I0807 16:21:00.055712 139944380065600 create_pet_tf_record.py:273] Reading from Pet dataset.
I0807 16:21:00.064566 139944380065600 create_pet_tf_record.py:288] 2576 training and 1104 validation examples.
I0807 16:21:00.064963 139944380065600 create_pet_tf_record.py:241] On image 0 of 2576
I0807 16:21:00.208813 139944380065600 create_pet_tf_record.py:241] On image 100 of 2576
I0807 16:21:00.348456 139944380065600 create_pet_tf_record.py:241] On image 200 of 2576
I0807 16:21:00.485131 139944380065600 create_pet_tf_record.py:241] On image 300 of 2576
I0807 16:21:00.633346 139944380065600 create_pet_tf_record.py:241] On image 400 of 2576
I0807 16:21:00.767741 139944380065600 create_pet_tf_record.py:241] On image 500 of 2576
W0807 16:21:00.865616 139944380065600 create_pet_tf_record.py:246] Could not find annotations/xmls/Egyptian_Mau_14.xml, ignoring example.
W0807 16:21:00.888195 139944380065600 create_pet_tf_record.py:246] Could not find annotations/xmls/saint_bernard_15.xml, ignoring ex

# 到 [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) 下載 ssd_mobilenet_v1_coco 預訓練模型權重

- 此範例以 ssd_mobilenet_v1 作參考，可以試用別的 (如 ssd_mobilenet_v2)

In [12]:
# 下載模型權重
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
# 解壓縮
!tar -xvf ssd_mobilenet_v1_coco_2018_01_28.tar.gz
!cp ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.* .

--2020-08-07 16:21:16--  http://storage.googleapis.com/download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.200.240, 172.217.160.80, 172.217.27.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.200.240|:80... connected.
HTTP request sent, awaiting response...200 OK
Length: 76541073 (73M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_coco_2018_01_28.tar.gz’

ssd_mobilenet_v1_co 100%[===================>]  73.00M  11.2MB/s    in 7.7s    

2020-08-07 16:21:24 (9.50 MB/s) - ‘ssd_mobilenet_v1_coco_2018_01_28.tar.gz’ saved [76541073/76541073]

ssd_mobilenet_v1_coco_2018_01_28/
ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.index
ssd_mobilenet_v1_coco_2018_01_28/checkpoint
ssd_mobilenet_v1_coco_2018_01_28/pipeline.config
ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.data-00000-of-00001
ssd_mobilenet_v1_coco_2018_01_28/model.ckpt.meta
ssd_mobilenet_v1_coco_2018_01_28

# 調整訓練設定及超參數

- 設定檔皆在 object_detection/samples/configs/ 裡，選擇對應的模型設定檔
- 此範例為了簡化，故用指令對設定檔作調整，實際上可以直接打開檔案作調整
- 一些必備的調整如
    - `PATH_TO_BE_CONFIGURED` 為調整檔案路徑位置
    - `mscoco_label_map.pbtxt` & `mscoco_train.record` & `mscoco_val.record` 為調整資料集
- 其他超參數也可以作調整，是需求而定

In [14]:
!cp /home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config .
# 修改檔案位置
!sed -i "s|/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎|\.|g" ssd_mobilenet_v1_coco.config
# 修改資料集
!sed -i "s|mscoco_label_map.pbtxt|object_detection/data/pet_label_map.pbtxt|g" ssd_mobilenet_v1_coco.config
!sed -i "s|mscoco_train.record|pet_faces_train.record|g" ssd_mobilenet_v1_coco.config
!sed -i "s|mscoco_val.record|pet_faces_val.record|g" ssd_mobilenet_v1_coco.config
# 由於 pets 最終只會產生 10 份 tfrecord, 因此這裏稍作修改
!sed -i "s|00100|00010|g" ssd_mobilenet_v1_coco.config

# 執行訓練

In [5]:
import sys
# sys.path.append(os.getcwd())
# %set_env PYTHONPATH=/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎\ /home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research:/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎 \/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research/slim

# !export PYTHONPATH=$PYTHONPATH:/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models

os.environ['PYTHONPATH'] += ":/models"
sys.path.append("/models")

# sys.path.append("/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/official")
from official.mnist import dataset

!python /home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎/models/research/object_detection/model_main.py \
 --logtostderr \
 --pipeline_config_path=ssd_mobilenet_v1_coco.config \
 --train_dir=train

ModuleNotFoundError: No module named 'official.mnist'

In [3]:
!pwd

/home/rico-li/Job/1st-DL-CVMarathon_Rico/電腦視覺深度學習基礎
